In [2]:
import datetime

import pandas as pd
from datetime import datetime

# Load raw medication data from Round
round_raw = pd.read_csv('data/Round Health 2021-12-31-12-49.csv')

# Clean round_raw
round_wrk = round_raw.copy()
# Remove irrelevant columns
round_wrk = round_wrk.drop(columns=['Start time', 'End time', 'Quantity', 'Timezone', 'Date'])

# Standardize strength values
round_wrk.Strength = round_wrk.Strength.replace('Lamotrigine', '', regex=True)
round_wrk.Strength = round_wrk.Strength.replace('mg', '', regex=True)
round_wrk.Strength = [float(x) for x in round_wrk.Strength]

# Convert dates from str to date
round_wrk['Taken On'] = [datetime.strptime(x, '%Y-%m-%d') if x != '--' else datetime.strptime('0001-01-01', '%Y-%m-%d') for x in round_wrk['Taken On']]
round_wrk['Taken'] = [0 if x == '--' else 1 for x in round_wrk['Taken At']]
round_wrk['Taken At'] = [datetime.strptime(x, '%I:%M %p').time() if x != '--' else datetime.strptime('01:00 AM', '%I:%M %p').time() for x in round_wrk['Taken At']]
round_wrk

,Name,Strength,Taken On,Taken At,Taken
0,Zoloft,25.0,2020-07-20 00:00:00,09:44:00,1
1,Zoloft,25.0,2020-07-21 00:00:00,08:47:00,1
2,Zoloft,25.0,2020-07-22 00:00:00,08:58:00,1
3,Zoloft,25.0,2020-07-23 00:00:00,08:33:00,1
4,Zoloft,25.0,2020-07-24 00:00:00,08:50:00,1
...,...,...,...,...,...
616,Lamictal,125.0,2021-12-29 00:00:00,09:22:00,1
617,Zoloft,50.0,2021-12-30 00:00:00,10:14:00,1
618,Lamictal,125.0,2021-12-30 00:00:00,10:14:00,1
619,Zoloft,50.0,2021-12-31 00:00:00,10:36:00,1


In [3]:
# Load mood data from Pixels
pixels_raw = pd.read_json('data/PixelsExport_2021-12-31.json')

# Remove columns
pixels_wrk = pixels_raw.copy()
pixels_wrk = pixels_wrk.drop(columns=['notes'])
pixels_wrk['date'] = [str(x) for x in pixels_wrk.date]
pixels_wrk

,mood,emotions,date
0,4,"[chill, stress, excitement, nerves, anger]",2021-01-03 00:00:00
1,3,"[tiredness, love, nerves, chill, optimism, fear]",2021-01-04 00:00:00
2,3,"[joy, emptiness, chill, excitement, tiredness]",2021-01-05 00:00:00
3,2,"[emptiness, tiredness, stress, chill, happines...",2021-01-06 00:00:00
4,1,"[remorse, excitement, tiredness, nerves, stres...",2021-01-07 00:00:00
...,...,...,...
358,3,[],2021-12-27 00:00:00
359,3,[],2021-12-28 00:00:00
360,4,[],2021-12-29 00:00:00
361,4,[],2021-12-30 00:00:00


In [4]:
import numpy as np

# Combine tables
round_wrk['date'] = [str(x) for x in round_wrk['Taken On']]
master = pixels_wrk.merge(round_wrk, how='left', on='date')
master.Taken[master.Taken != 1.0] = [0]
master = master.drop(columns=['Taken On'])
master['Taken At'] = [x if type(x) is not float else '00:00:00' for x in master['Taken At']]
master

/var/folders/zl/f1r1src11s71p47cjq22tpbw0000gn/T/ipykernel_62086/2048732795.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master.Taken[master.Taken != 1.0] = [0]


,mood,emotions,date,Name,Strength,Taken At,Taken
0,4,"[chill, stress, excitement, nerves, anger]",2021-01-03 00:00:00,Zoloft,50.0,11:33:00,1.0
1,3,"[tiredness, love, nerves, chill, optimism, fear]",2021-01-04 00:00:00,Zoloft,50.0,12:02:00,1.0
2,3,"[joy, emptiness, chill, excitement, tiredness]",2021-01-05 00:00:00,Zoloft,50.0,12:07:00,1.0
3,2,"[emptiness, tiredness, stress, chill, happines...",2021-01-06 00:00:00,Zoloft,50.0,11:18:00,1.0
4,1,"[remorse, excitement, tiredness, nerves, stres...",2021-01-07 00:00:00,Zoloft,50.0,11:06:00,1.0
...,...,...,...,...,...,...,...
463,4,[],2021-12-29 00:00:00,Lamictal,125.0,09:22:00,1.0
464,4,[],2021-12-30 00:00:00,Zoloft,50.0,10:14:00,1.0
465,4,[],2021-12-30 00:00:00,Lamictal,125.0,10:14:00,1.0
466,4,[],2021-12-31 00:00:00,Zoloft,50.0,10:36:00,1.0


In [5]:
master.describe()

,mood,Strength,Taken
count,468.000000,452.000000,468.000000
mean,3.089744,55.586283,0.965812
std,0.921119,17.947301,0.181906
min,1.000000,25.000000,0.000000
25%,3.000000,50.000000,1.000000
50%,3.000000,50.000000,1.000000
75%,4.000000,50.000000,1.000000
max,5.000000,125.000000,1.000000


In [6]:
# Create additional time data

master['dt_val'] = [pd.to_datetime(x) for x in master.date]
master['week'] = [x.isocalendar()[1] for x in master.dt_val]
master['month'] = [x.month for x in master.dt_val]
master['day_of_week_num'] = [x.isocalendar()[2] for x in master.dt_val]

weekday_dict = {'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7}
weekday_dict_corrected = dict((v,k) for k,v in weekday_dict.items())


def get_weekday(weekday_num):
    return weekday_dict_corrected[weekday_num]


master['day_of_week'] = [get_weekday(x) for x in master['day_of_week_num']]

def day(date):
    return date.day_of_year


def get_season(date):
    if day(date) < 79:
        return 'Winter'
    elif 79 <= day(date) < 172:
        return 'Spring'
    elif 172 <= day(date) < 265:
        return 'Summer'
    elif 265 <= day(date) < 355:
        return 'Fall'
    else:
        return 'Winter'


master['season'] = [get_season(x) for x in master.dt_val]
master = master.drop(columns=['day_of_week_num', 'date'])

master['Name'] = [x if type(x) is str else 'Not Taken' for x in master.Name]

In [7]:
def time_to_numeric(t):
    ts = str(t)
    (h, m, s) = ts.split(':')
    result = int(h) * 3600 + int(m) * 60 + int(s)
    return result

master['TimeNumeric'] =[time_to_numeric(x) for x in master['Taken At']]
master

,mood,emotions,Name,Strength,Taken At,Taken,dt_val,week,month,day_of_week,season,TimeNumeric
0,4,"[chill, stress, excitement, nerves, anger]",Zoloft,50.0,11:33:00,1.0,2021-01-03,53,1,Saturday,Winter,41580
1,3,"[tiredness, love, nerves, chill, optimism, fear]",Zoloft,50.0,12:02:00,1.0,2021-01-04,1,1,Sunday,Winter,43320
2,3,"[joy, emptiness, chill, excitement, tiredness]",Zoloft,50.0,12:07:00,1.0,2021-01-05,1,1,Monday,Winter,43620
3,2,"[emptiness, tiredness, stress, chill, happines...",Zoloft,50.0,11:18:00,1.0,2021-01-06,1,1,Tuesday,Winter,40680
4,1,"[remorse, excitement, tiredness, nerves, stres...",Zoloft,50.0,11:06:00,1.0,2021-01-07,1,1,Wednesday,Winter,39960
...,...,...,...,...,...,...,...,...,...,...,...,...
463,4,[],Lamictal,125.0,09:22:00,1.0,2021-12-29,52,12,Tuesday,Winter,33720
464,4,[],Zoloft,50.0,10:14:00,1.0,2021-12-30,52,12,Wednesday,Winter,36840
465,4,[],Lamictal,125.0,10:14:00,1.0,2021-12-30,52,12,Wednesday,Winter,36840
466,4,[],Zoloft,50.0,10:36:00,1.0,2021-12-31,52,12,Thursday,Winter,38160


In [8]:
import numpy as np
all_moods = []
for x in master.emotions:
    for y in x:
        all_moods.append(y)
(mood, ct) = np.unique(all_moods, return_counts=True)
frequencies = np.asarray((mood, ct)).T
freq_df = pd.DataFrame(frequencies, columns=['Emotion', 'Count'])
freq_df = freq_df.convert_dtypes()
freq_df.Count = freq_df.Count.astype('int')
freq_df

,Emotion,Count
0,anger,78
1,anxiety,105
2,chill,152
3,emptiness,108
4,excitement,134
5,fear,61
6,happiness,89
7,joy,120
8,love,74
9,nerves,101


In [9]:
# mood score
(mood, score) = (frequencies[:,0],[-1,-1,1,-1,1,-1,1,1,1,-1,1,-1,-1,-1,-1])
mood_scores = np.asarray((mood, score), dtype=object).T
scores_dict = dict(list(map(tuple, mood_scores.reshape((15, 2)))))
score = []
for x in master.emotions:
    if not x:
        score.append(0)
        continue
    n = []
    for y in x:
        n.append(scores_dict[y])
    score.append(sum(n)/len(n))
master['emotion score'] = score
master[0:5]

,mood,emotions,Name,Strength,Taken At,Taken,dt_val,week,month,day_of_week,season,TimeNumeric,emotion score
0,4,"[chill, stress, excitement, nerves, anger]",Zoloft,50.0,11:33:00,1.0,2021-01-03,53,1,Saturday,Winter,41580,-0.200000
1,3,"[tiredness, love, nerves, chill, optimism, fear]",Zoloft,50.0,12:02:00,1.0,2021-01-04,1,1,Sunday,Winter,43320,0.000000
2,3,"[joy, emptiness, chill, excitement, tiredness]",Zoloft,50.0,12:07:00,1.0,2021-01-05,1,1,Monday,Winter,43620,0.200000
3,2,"[emptiness, tiredness, stress, chill, happines...",Zoloft,50.0,11:18:00,1.0,2021-01-06,1,1,Tuesday,Winter,40680,-0.555556
4,1,"[remorse, excitement, tiredness, nerves, stres...",Zoloft,50.0,11:06:00,1.0,2021-01-07,1,1,Wednesday,Winter,39960,-0.750000


In [10]:
# Create variance table
mood_weekly = master.groupby('week')['mood'].mean().reset_index()
mood_weekly['avg'] = mood_weekly.mood.copy()
mood_weekly = mood_weekly.drop(columns=['mood'])

In [11]:
mood_weekly['variance'] = master.groupby('week')['mood'].var().reset_index().mood
mood_weekly['habit_score'] = master.groupby('week')['Taken'].mean().reset_index().Taken
mood_weekly['time_variance'] = master.groupby('week')['TimeNumeric'].var().reset_index().TimeNumeric/10000000
mood_weekly['emotion_variance'] = master.groupby('week')['emotion score'].var().reset_index()['emotion score']
mood_weekly['emotion_mean'] = master.groupby('week')['emotion score'].mean().reset_index()['emotion score']
mood_weekly

,week,avg,variance,habit_score,time_variance,emotion_variance,emotion_mean
0,1,2.857143,1.142857,1.000000,1.713051,0.143702,-0.142063
1,2,3.000000,1.666667,1.000000,1.909680,0.185619,-0.129226
2,3,2.571429,0.952381,0.857143,18.936206,0.051967,0.067033
3,4,2.857143,1.142857,0.857143,21.498051,0.171760,0.249660
4,5,2.857143,0.809524,1.000000,2.696451,0.161071,-0.173810
5,6,2.714286,1.571429,1.000000,3.631766,0.244462,-0.379365
6,7,3.000000,0.444444,1.000000,1.641300,0.147934,-0.184444
7,8,3.142857,0.809524,1.000000,1.798731,0.109998,-0.037415
8,9,3.714286,0.571429,1.000000,1.881566,0.119922,0.153741
9,10,3.000000,1.333333,1.000000,3.114480,0.183185,-0.004082


In [12]:
mood_monthly = master.groupby('month')['mood'].mean().reset_index()
mood_monthly['avg'] = mood_monthly.mood.copy()
mood_monthly = mood_monthly.drop(columns=['mood'])
mood_monthly['variance'] = master.groupby('month')['mood'].var().reset_index().mood
mood_monthly['habit_score'] = master.groupby('month')['Taken'].mean().reset_index().Taken
mood_monthly['time_variance'] = master.groupby('month')['TimeNumeric'].var().reset_index().TimeNumeric/10000000
mood_monthly['emotion_variance'] = master.groupby('month')['emotion score'].var().reset_index()['emotion score']
mood_monthly['emotion_mean'] = master.groupby('month')['emotion score'].mean().reset_index()['emotion score']
mood_monthly

,month,avg,variance,habit_score,time_variance,emotion_variance,emotion_mean
0,1,2.862069,1.123153,0.931034,12.223398,0.145848,0.004063
1,2,2.935484,0.795699,1.000000,2.293202,0.161349,-0.192857
2,3,3.000000,1.200000,0.967742,5.470017,0.176154,-0.130024
3,4,3.031250,0.740927,0.875000,17.587042,0.112579,-0.177523
4,5,3.142857,0.831933,0.914286,18.922099,0.152691,-0.051519
5,6,3.212121,0.922348,0.909091,16.513566,0.108874,-0.155029
6,7,3.290323,0.612903,0.967742,6.959920,0.088373,-0.028923
7,8,3.181818,0.965909,0.969697,9.492813,0.133468,-0.060029
8,9,3.166667,0.557471,0.966667,7.274680,0.044020,-0.061126
9,10,3.114754,1.236612,1.000000,1.684398,0.000000,0.000000


In [13]:
mood_seasons = master.groupby('season')['mood'].mean().reset_index()
mood_seasons['avg'] = mood_seasons.mood.copy()
mood_seasons = mood_seasons.drop(columns=['mood'])
mood_seasons['variance'] = master.groupby('season')['mood'].var().reset_index().mood
mood_seasons['habit_score'] = master.groupby('season')['Taken'].mean().reset_index().Taken
mood_seasons['time_variance'] = master.groupby('season')['TimeNumeric'].var().reset_index().TimeNumeric/10000000
mood_seasons['emotion_variance'] = master.groupby('season')['emotion score'].var().reset_index()['emotion score']
mood_seasons['emotion_mean'] = master.groupby('season')['emotion score'].mean().reset_index()['emotion score']
mood_seasons

,season,avg,variance,habit_score,time_variance,emotion_variance,emotion_mean
0,Fall,3.082353,0.892586,1.000000,3.080193,0.000000,0.000000
1,Spring,3.049020,0.878761,0.892157,18.558451,0.125515,-0.135692
2,Summer,3.263158,0.727884,0.968421,8.515748,0.099630,-0.056155
3,Winter,2.980198,0.839604,0.980198,4.980739,0.135441,-0.079479


In [15]:
import plotly.express as px
varianceXweek = px.scatter(mood_weekly, x='week', y='variance', trendline='ols')
varianceXweek.show()

In [16]:
box = px.box(master, x='season', y='mood', color='Name')
box.show()

In [17]:
import researchpy as rp
import scipy.stats as stats

rp.ttest(group1 = master.mood[master.season == 'Summer'], group1_name='Summer', group2 = master.mood[master.season == 'Winter'], group2_name='Winter')

(   Variable      N      Mean        SD        SE  95% Conf.  Interval
 0    Summer   95.0  3.263158  0.853161  0.087532   3.089360  3.436956
 1    Winter  101.0  2.980198  0.916299  0.091175   2.799309  3.161087
 2  combined  196.0  3.117347  0.895290  0.063949   2.991226  3.243468,
                 Independent t-test   results
 0  Difference (Summer - Winter) =     0.2830
 1            Degrees of freedom =   194.0000
 2                             t =     2.2339
 3         Two side test p value =     0.0266
 4        Difference < 0 p value =     0.9867
 5        Difference > 0 p value =     0.0133
 6                     Cohen's d =     0.3193
 7                     Hedge's g =     0.3180
 8                 Glass's delta =     0.3317
 9                   Pearson's r =     0.1584)

In [18]:
rp.ttest(group1 = master.mood[master.Taken == 1.0], group1_name='Taken', group2 = master.mood[master.Taken == 0], group2_name='Not Taken')

(    Variable      N      Mean        SD        SE  95% Conf.  Interval
 0      Taken  452.0  3.086283  0.917476  0.043154   3.001474  3.171092
 1  Not Taken   16.0  3.187500  1.046821  0.261705   2.629689  3.745311
 2   combined  468.0  3.089744  0.921119  0.042579   3.006074  3.173413,
                   Independent t-test   results
 0  Difference (Taken - Not Taken) =    -0.1012
 1              Degrees of freedom =   466.0000
 2                               t =    -0.4316
 3           Two side test p value =     0.6662
 4          Difference < 0 p value =     0.3331
 5          Difference > 0 p value =     0.6669
 6                       Cohen's d =    -0.1098
 7                       Hedge's g =    -0.1096
 8                   Glass's delta =    -0.1103
 9                     Pearson's r =     0.0200)

In [19]:
varXmood = px.scatter(mood_weekly, x='avg', y='variance', trendline='ols')
varXmood.show()

In [23]:
varianceXhs = px.scatter(mood_weekly, x='habit_score', y='variance', trendline='ols')
varianceXhs.show()

In [24]:
averageXhs = px.scatter(mood_weekly, x='habit_score', y='avg', trendline='ols')
averageXhs.show()

In [25]:
varianceXtimevariance = px.scatter(mood_weekly, x='time_variance', y='avg', trendline='ols')
varianceXtimevariance.show()

In [26]:
fig = px.scatter(mood_monthly, x='month', y='variance', trendline='ols')
fig.show()

In [27]:
fig1 = px.scatter(mood_monthly, x='month', y='avg', trendline='lowess')
fig1.show()

In [28]:
fig2 = px.scatter(mood_monthly, x='avg', y='variance', trendline='ols')
fig2.show()

In [29]:
fig5 = px.scatter(mood_monthly, x='month', y='habit_score', trendline='ols')
fig5.show()

In [30]:
emotion_freq = px.bar(freq_df, x='Emotion', y='Count', color='Count', color_continuous_scale='ice')
emotion_freq.update_layout(xaxis={'categoryorder':'total descending'})